# 검색 및 관련성 평가 (Retrieve & Grade)

이 노트북에서는 **RAG 시스템에서 검색 결과의 관련성을 평가**하는 방법을 배웁니다.

## RAG 평가의 중요성

```
┌────────────────────────────────────────────────────────────────────┐
│                    RAG 평가가 필요한 이유                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   RAG 시스템:                                                      │
│   질문 → 검색 → 문서들 → LLM → 답변                               │
│                                                                    │
│   문제:                                                            │
│   검색된 문서가 질문과 관련 없으면?                                │
│   → LLM이 잘못된 정보로 답변                                       │
│   → 환각(Hallucination) 발생                                       │
│                                                                    │
│   해결:                                                            │
│   검색 결과의 관련성을 LLM으로 평가!                               │
│   → 관련 없는 문서 필터링                                          │
│   → 필요시 재검색 또는 웹 검색                                     │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    검색 + 평가 흐름                                 │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   질문: "2024년 LangGraph 에이전트 사례는?"                        │
│     │                                                              │
│     ▼                                                              │
│   ┌──────────────┐                                                 │
│   │   Retriever  │  → 문서 4개 검색                                │
│   └──────┬───────┘                                                 │
│          │                                                         │
│          ▼                                                         │
│   ┌──────────────┐                                                 │
│   │    Grader    │  → 각 문서 관련성 평가 (yes/no)                │
│   │ (LLM 평가기) │                                                 │
│   └──────┬───────┘                                                 │
│          │                                                         │
│   관련 문서만 선택                                                  │
│          │                                                         │
│          ▼                                                         │
│   ┌──────────────┐                                                 │
│   │   Generator  │  → 관련 문서로 답변 생성                        │
│   └──────────────┘                                                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langchain-community beautifulsoup4 tiktoken

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. 문서 로드 및 인덱싱

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings

# 블로그 게시물 URL
urls = [
    "https://blog.langchain.dev/top-5-langgraph-agents-in-production-2024/",
    "https://blog.langchain.dev/langchain-state-of-ai-2024/",
    "https://blog.langchain.dev/introducing-ambient-agents/",
]

# 문서 로드
print("=== 문서 로드 중 ===")
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
print(f"로드된 문서: {len(docs_list)}개")

In [ ]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, 
    chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)
print(f"분할된 청크: {len(doc_splits)}개")

In [ ]:
# 벡터 스토어 생성
embeddings = OllamaEmbeddings(model='nomic-embed-text')

vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

print("✅ 벡터 스토어 및 Retriever 생성 완료")

# 3. 검색 테스트

In [ ]:
# 검색 테스트
question = "2024년에 프로덕션 환경에서 사용된 LangGraph 에이전트 2개는 무엇인가요?"

results = retriever.invoke(question)

print(f"=== 검색 결과 ({len(results)}개) ===")
for i, doc in enumerate(results):
    print(f"\n[문서 {i+1}]")
    print(f"내용: {doc.page_content[:200]}...")

# 4. 관련성 평가 스키마 정의

In [ ]:
from pydantic import BaseModel, Field

class GradeDocuments(BaseModel):
    """검색된 문서의 관련성 평가 결과"""
    
    binary_score: str = Field(
        description="문서가 질문과 관련이 있으면 'yes', 없으면 'no'"
    )

print("✅ GradeDocuments 스키마 정의 완료")

# 5. 관련성 평가기 (Grader) 생성

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# LLM 설정 (Structured Output)
llm = ChatOllama(model='llama3.2', temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# 평가 프롬프트
system = """당신은 사용자 질문에 대한 검색된 문서의 관련성을 평가하는 채점자입니다.
문서에 질문과 관련된 키워드나 의미가 포함되어 있다면 관련성이 있다고 평가하세요.
문서가 질문과 관련이 있는지 여부를 나타내는 'yes' 또는 'no'로 평가해 주세요."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "검색된 문서: \n\n {document} \n\n 사용자 질문: {question}"),
    ]
)

# 평가 체인
retrieval_grader = grade_prompt | structured_llm_grader

print("✅ 관련성 평가기 생성 완료")

# 6. 검색 결과 평가

In [ ]:
# 각 문서 평가
print("=== 관련성 평가 ===")
print(f"질문: {question}\n")

relevant_docs = []

for i, doc in enumerate(results):
    result = retrieval_grader.invoke({
        "question": question, 
        "document": doc.page_content
    })
    
    status = "✅ 관련" if result.binary_score == 'yes' else "❌ 무관"
    print(f"[문서 {i+1}] {status}")
    print(f"  내용: {doc.page_content[:100]}...")
    print()
    
    if result.binary_score == 'yes':
        relevant_docs.append(doc)

print(f"\n관련 문서: {len(relevant_docs)}개 / 전체: {len(results)}개")

# 7. 관련 문서로 답변 생성

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 답변 생성 프롬프트
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문에 답변하는 어시스턴트입니다. 주어진 문서를 바탕으로 답변하세요."),
    ("human", "문서:\n{context}\n\n질문: {question}")
])

# 컨텍스트 생성
context = "\n\n".join([doc.page_content for doc in relevant_docs])

# 답변 생성
answer_chain = answer_prompt | llm
answer = answer_chain.invoke({"context": context, "question": question})

print("=== 최종 답변 ===")
print(answer.content)

---

## 정리: 검색 관련성 평가

### 핵심 코드

```python
# 1. 평가 스키마
class GradeDocuments(BaseModel):
    binary_score: str = Field(description="'yes' 또는 'no'")

# 2. Structured Output LLM
grader = llm.with_structured_output(GradeDocuments)

# 3. 평가 체인
retrieval_grader = grade_prompt | grader

# 4. 각 문서 평가
for doc in docs:
    result = retrieval_grader.invoke({"question": q, "document": doc})
    if result.binary_score == 'yes':
        relevant_docs.append(doc)
```

### 평가 활용

| 평가 결과 | 다음 행동 |
|----------|----------|
| 관련 문서 있음 | 답변 생성 |
| 관련 문서 없음 | 재검색 또는 웹 검색 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(temperature=0)

# 변경
embeddings = OllamaEmbeddings(model='nomic-embed-text')
llm = ChatOllama(model='llama3.2', temperature=0)
```

## 다음 단계

**LangSmith**를 사용한 체계적인 평가 방법을 배웁니다. (03-06번 노트북)